<a href="https://colab.research.google.com/github/bhupennitkit20/ImageAI/blob/master/Copy_of_Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import math
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
# %matplotlib inline
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize

data = pd.read_csv("/content/mapping.csv")
test = pd.read_csv('/content/testing.csv')

X = []
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

from keras.utils import np_utils
train_y = np_utils.to_categorical(data.Class)
test_y = np_utils.to_categorical(test.Class)

image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')
test_image = preprocess_input(test_image, mode='tf')

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
test_image = base_model.predict(test_image)

X_train = X_train.reshape(155, 7*7*512)
X_valid = X_valid.reshape(67, 7*7*512)
test_image = test_image.reshape(36, 7*7*512)

train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
test_image = test_image/test_image.max()

model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(3, activation='softmax'))            # output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Class), data.Class)  # computing weights of different classes

from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss

model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)

model.load_weights("weights.best.hdf5")

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

scores = model.evaluate(test_image, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))




Train on 155 samples, validate on 67 samples
Epoch 1/10
155/155 [==============================] - 2s 15ms/step - loss: 1.3905 - accuracy: 0.3742 - val_loss: 1.0091 - val_accuracy: 0.5373

Epoch 00001: val_loss improved from inf to 1.00907, saving model to weights.best.hdf5
Epoch 2/10
155/155 [==============================] - 2s 13ms/step - loss: 1.0338 - accuracy: 0.5226 - val_loss: 0.8895 - val_accuracy: 0.7015

Epoch 00002: val_loss improved from 1.00907 to 0.88951, saving model to weights.best.hdf5
Epoch 3/10
155/155 [==============================] - 2s 13ms/step - loss: 1.1049 - accuracy: 0.3742 - val_loss: 0.7938 - val_accuracy: 0.5373

Epoch 00003: val_loss improved from 0.88951 to 0.79382, saving model to weights.best.hdf5
Epoch 4/10
155/155 [==============================] - 2s 14ms/step - loss: 0.9457 - accuracy: 0.5871 - val_loss: 0.7136 - val_accuracy: 0.5373

Epoch 00004: val_loss improved from 0.79382 to 0.71360, saving model to weights.best.hdf5
Epoch 5/10
155/155 [===

In [0]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
# %matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images


data = pd.read_csv('/content/mapping.csv')

'''# show a image
img = plt.imread('frame0.jpg')   # reading image using its name
plt.imshow(img)

# Print data of the csv file
data = pd.read_csv('mapping.csv')     # reading the csv file
data.head()      # printing first five rows of the file'''


X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array


y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes


image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)


from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')      # preprocessing the input data


from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer


X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

X_train = X_train.reshape(155, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(67, 7*7*512)

train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()


# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(3, activation='softmax'))    # output layer
model.summary()


# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# iii. Training the model
model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid))

test = pd.read_csv('test.csv')
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

# preprocessing the images
test_image = preprocess_input(test_image, mode='tf')

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(36, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

predictions = model.predict_classes(test_image)

print("No of appearances of both is", predictions[predictions==2].shape[0], "times")
print("No of appearances of Dog is", predictions[predictions==0].shape[0], "times")
print("No of appearances of Human is", predictions[predictions==1].shape[0], "times")



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________
Train on 155 samples, validate on 67 samples
Epoch 1/10
155/155 [==============================] - 2s 13ms/step - loss: 0.8286 - accuracy: 0.6387 - val_loss: 0.2304 - val_accuracy: 0.9254
Epoch 2/10
155/155 [==============================] - 2s 13ms/step - loss: 0.1688 - accuracy: 0.9419 - val_loss: 0.1262 - val_accuracy: 0.9851
Epoch 3/10
155/155 [==============================] - 2s 13ms/step - loss: 0.0379 - accuracy: 1.0000 - val_loss: 0.0216 - val_accuracy: 1.0000
Epoch 4/10
155/155 [==========

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

07_structured_data.ipynb	   Sample Excel file.xlsx
BigQuery recipes		   script.ipynb
Colab Notebooks			   TFGan tutorial in Colab.txt
Copy of nima colab.ipynb	   to_upload (1).ipynb
created.txt			   to_upload (2).ipynb
Exported DataFrame sheet.gsheet    to_upload (3).ipynb
foo.txt				   to_upload.ipynb
Pickle + Drive FUSE example.ipynb  variables.pickle
Sample Excel file.gsheet
